In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit

# fix for 'package not found' when installing in Anaconda environment
if 'google.colab' not in str(get_ipython()):
    import pip
    pip.main(['install', 'xgboost'])

if 'google.colab' in str(get_ipython()):
  !rm util.py
  !wget https://raw.githubusercontent.com/marco-mazzoli/progetto-tesi/master/util.py

from xgboost import XGBRegressor
from util import select_relevant_rows, select_attributes, read_movement_data, read_multiple_csv,download_updated_mobility_data, download_updated_mobility_data, time_series_cross_validation, train_and_predict, select_time_slot

local_region_path = r'../COVID-19/dati-regioni/dpc-covid19-ita-regioni.csv'
remote_region_path = r'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

regions_frame = pd.read_csv(remote_region_path)

region_focus = 'Emilia-Romagna'
attribute_focus = 'denominazione_regione'

region_focus_data = select_relevant_rows(
    regions_frame,
    attribute_focus,
    region_focus
    )

frame_interesting_columns = select_attributes(region_focus_data, [
    'data',
    'ricoverati_con_sintomi',
    'terapia_intensiva',
    'totale_ospedalizzati',
    'variazione_totale_positivi',
    'nuovi_positivi',
    'deceduti',
    'tamponi',
    'ingressi_terapia_intensiva'
    ])

frame_interesting_columns = pd.DataFrame(frame_interesting_columns)
frame_interesting_columns['data'] = pd.to_datetime(frame_interesting_columns['data'])
frame_interesting_columns['data'] = frame_interesting_columns['data'].dt.strftime(r'%Y-%m-%d')
frame_interesting_columns = frame_interesting_columns.fillna(0)
frame_interesting_columns.rename(columns={'data': 'date'}, inplace=True)
frame_interesting_columns.set_index('date',inplace=True)

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [2]:
column_to_predict = 'terapia_intensiva'

univariate_series = frame_interesting_columns[[column_to_predict]]

In [10]:
n_splits = 10
test_size = 10

tscv = TimeSeriesSplit(n_splits=n_splits,test_size=test_size)

for train_index, test_index in tscv.split(univariate_series):
    X_train = train.iloc[train_index]
    y_train = train.iloc[test_index]

    regressor = XGBRegressor(
        objective='reg:squarederror',
        n_estimators=1000
        )

    regressor.fit(X_train,y_train)

    pred = regressor.predict(X_test)
    pred = pd.DataFrame(pred).set_index(test.index)

    print('MAE: ', mean_absolute_error(pred, test))
    print('train interval: ' + str(train.index[0]) + ' - ' + str(train.index[-1]))
    print('test interval: ' + str(test.index[0]) + ' - ' + str(test.index[-1]))
    pyplot.title('predictions')
    pyplot.plot(test,label='real')
    pyplot.plot(pred,label='predicted')
    pyplot.ylabel(column_to_predict)
    pyplot.xticks(rotation=30)
    pyplot.legend(loc='best')
    pyplot.show()

TypeError: '(slice(0, 411, None), slice(None, None, None))' is an invalid key